In [14]:
import pandas as pd
import numpy as np
from sklearn import svm
# sklearn下cross_validation：交叉验证
from sklearn import model_selection
#parse_dates=第0列解析为日期， index_col= 用作行索引的列编号）
data =pd.read_csv('train_data.csv',parse_dates=[0],index_col=0)
#DataFrame.sort_index(axis=0 (按0列排), ascending=True（升序）,
#inplace=False（排序后是否覆盖原数据））data 按照时间升序排列
data.sort_index(0,ascending=True,inplace=True)

In [15]:
data.shape

(4865, 11)

In [16]:
#dayfeature：选取4856-300天的数据
#featurenum：选取的5个特征*天数
#x：记录150天的5个特征值 y：记录涨或者跌
dayfeature=4856-300
featurenum =5*dayfeature
# 即150*5， +1是将当天的开盘价引入作为一条特征数据
x=np.zeros((data.shape[0]-dayfeature,featurenum+1))
y=np.zeros((data.shape[0]-dayfeature))

In [21]:
for i in range(0,data.shape[0]-dayfeature):
    x[i,0:featurenum]=np.array(data[i:i+dayfeature]\
                               [['close','high',   'low',u'open','vol']]).reshape((1,featurenum))
    x[i,featurenum]=data.loc[i+dayfeature]['open']
for i in range(0,data.shape[0]-dayfeature):
    if data.loc[i+dayfeature]['close']>=data.loc[i+dayfeature]['open']:
        y[i]=1
    else:
        y[i]=0
        #如果当天收盘价高于开盘价，y[i]=1代表涨，0代表跌
#创建SVM并进行交叉验证
clf =svm.SVC(kernel='sigmoid')
result =[]
for i in range(5):
    #x和y的验证集和测试集，切分80 - 20 % 的测试集
    x_train,x_test,y_train,y_test =\
        model_selection.train_test_split(x,y,test_size=0.2)
    #训练数据进行训练
    clf.fit(x_train,y_train)
    #将预测数据和测试集的验证数据比对
    result.append(np.mean(y_test ==clf.predict(x_test)))
print("svm classifier accuacy:")
print(result)

svm classifier accuacy:
[0.5161290322580645, 0.5806451612903226, 0.45161290322580644, 0.6129032258064516, 0.6451612903225806]


In [ ]:
#经手动比对，为sigmoid时效果最好，五次实验交叉验证准确率仅为0.4-0.6